In [41]:
import requests
import pandas as pd

from io import StringIO


In [48]:
base_url = "http://127.0.0.1:8000/"
market_data_endpoint = base_url + "market-data/"

In [ ]:
resp = requests.get(market_data_endpoint + "historical?tickers=TSLA,AMZN&period=1d&interval=60m").json()

res = {}
for ticker, df_json in resp.items():
    res[ticker] = pd.read_json(
        StringIO(df_json),
        orient='index',
        convert_dates=True
    )

res

{'TSLA':                                 Close        High         Low        Open  \
 2025-08-13 13:30:00+00:00  345.089905  348.980011  340.239990  341.880005   
 2025-08-13 14:30:00+00:00  340.785004  345.562408  338.549988  345.100006   
 2025-08-13 15:30:00+00:00  341.300110  342.329987  340.200012  340.815002   
 2025-08-13 16:30:00+00:00  341.505005  341.659912  339.250000  341.299988   
 2025-08-13 17:30:00+00:00  340.225006  341.819794  338.230011  341.505005   
 2025-08-13 18:30:00+00:00  340.720001  341.454987  339.200012  340.260010   
 2025-08-13 19:30:00+00:00  339.339996  341.250000  339.200012  340.760010   
 
                              Volume  
 2025-08-13 13:30:00+00:00  23273591  
 2025-08-13 14:30:00+00:00  12224375  
 2025-08-13 15:30:00+00:00   6451030  
 2025-08-13 16:30:00+00:00   6283472  
 2025-08-13 17:30:00+00:00   6884780  
 2025-08-13 18:30:00+00:00   5159534  
 2025-08-13 19:30:00+00:00   4064061  ,
 'AMZN':                                 Close       

In [49]:
resp = requests.get(market_data_endpoint + "prices?tickers=TSLA,AMZN").json()
resp

{'TSLA': {'Datetime': '2025-08-13T15:59:00-04:00',
  'Open': 339.75,
  'High': 339.80999755859375,
  'Low': 339.20001220703125,
  'Close': 339.3399963378906,
  'Volume': 713716,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 'AMZN': {'Datetime': '2025-08-13T15:59:00-04:00',
  'Open': 224.6699981689453,
  'High': 224.6699981689453,
  'Low': 224.49000549316406,
  'Close': 224.55499267578125,
  'Volume': 751311,
  'Dividends': 0.0,
  'Stock Splits': 0.0}}